In [5]:
import json
import datetime
import os
import pandas as pd

In [54]:
datfiles=os.listdir('data/')

In [ ]:
def convert_timestring(x):
    x=x[:-6]+''.join(x[-6:].split(':'))
    return datetime.strptime(x,'%Y-%m-%dT%H:%M:%S%z')
    
def convert_timestring_utc(x):
    x=x[:-6]+''.join(x[-6:].split(':'))
    return datetime.strptime(x,'%Y-%m-%dT%H:%M:%S%z').astimezone(tz=datetime.timezone.utc)

In [55]:
order_dict={}
line_item_dict={}
user_dict={}

for fn in datfiles:
    with open('data/'+fn,'r') as f:
        print(fn)
        data=json.load(f)
        orders=data['orders']
        for order in orders:
            order_id=order['id']
            line_item_list=order['line_items']
            line_item_total=len(line_item_list)
            if order_id in order_dict:
                print('Error: Duplicative order_id: '+order_id)
                continue
            else:
                order_dict[order_id]={}
                order_dict[order_id]['line_item_total']=line_item_total
            
            for line_item in line_item_list:
                line_item_id=line_item['id']
                if line_item_id in line_item_dict:
                    print('Error: Duplicative line_item_id: '
                          +line_item_id+', order_id: '+order_id)
                    continue
                else:
                    line_item_dict[line_item_id]={k:v for k,v in line_item.items() if k!='id'}
                    line_item_dict[line_item_id]['order_id']=order_id
                    
            user_id=order['user_id']
            if user_id not in user_dict:
                user_dict[user_id]={k: v for k, v in order.items() if k in }
                user_dict[user_id]['order_id']=order_id

2017-10-30.json
97171
98171
97170
98170
97169
98169
97168
98168
97167
98167
97166
98166
97165
98165
97164
98164
97163
98163
97162
98162
97161
98161
97160
98160
97159
98159
97158
98158
97157
98157
97156
98156
97155
98155
97154
98154
97153
98153
97151
98151
97150
98150
97149
98149
97148
98148
97147
98147
97146
98146
97145
98145
97144
98144
2017-12-30.json
122647
123647
122646
123646
122645
123645
122644
123644
122643
123643
122642
123642
122641
123641
122640
123640
122639
123639
122638
123638
122637
123637
122636
123636
122635
123635
122634
123634
122633
123633
122632
123632
122631
123631
122630
123630
122629
123629
122628
123628
122627
123627
122626
123626
122625
123625
122624
123624
122623
123623
122622
123622
122621
123621
122620
123620
122619
123619
122618
123618
122617
123617
122616
123616
122615
123615
122614
123614
122613
123613
122612
123612
122611
123611
122610
123610
122609
123609
122608
123608
122607
123607
122606
123606
122605
123605
122604
123604
122603
123603
122602
123602


In [44]:

order_df=pd.DataFrame.from_dict(order_dict,orient='index')

In [45]:

line_item_df=pd.DataFrame.from_dict(line_item_dict,orient='index')

In [ ]:
variables to reformat:
    order and order_number are always 1000 apart
    closed_at
    created_at
    updated_at
    cancelled_at

In [ ]:
ignore variables that are mostly null

In [37]:
{k: v for k, v in line_item.items() if k!='id'}

{'product_id': 9086871491, 'quantity': 1, 'variant_id': 31894084547}

In [42]:
{k: v for k, v in line_item.items() if k not in set(['id','variant_id'])}

{'product_id': 9086871491, 'quantity': 1}

In [43]:
{k: v for k, v in line_item.items() if k in set(['id','variant_id'])}

{'id': 23337631747, 'variant_id': 31894084547}

In [46]:
{k: v for k, v in order.items() if k in set(['id','variant_id'])}

{'id': 11674779651}

In [48]:
order.items()

dict_items([('id', 11674779651), ('email', 'censored@censored.com'), ('closed_at', None), ('created_at', '2017-10-30T18:47:09-04:00'), ('updated_at', '2017-10-30T18:47:54-04:00'), ('number', 97144), ('note', None), ('token', 'censored'), ('gateway', 'shopify_payments'), ('test', False), ('total_price', '156.78'), ('subtotal_price', '144.0'), ('total_weight', 6), ('total_tax', '12.78'), ('taxes_included', False), ('currency', 'USD'), ('financial_status', 'paid'), ('confirmed', True), ('total_discounts', '0.00'), ('total_line_items_price', '144.0'), ('cart_token', 'censored'), ('buyer_accepts_marketing', False), ('name', 'GLO38-1966900000'), ('referring_site', None), ('landing_site', None), ('cancelled_at', None), ('cancel_reason', None), ('total_price_usd', '156.78'), ('checkout_token', 'censored'), ('reference', None), ('user_id', 110420867), ('location_id', 371291), ('source_identifier', '371291-38-19669'), ('source_url', None), ('processed_at', '2017-10-30T18:47:09-04:00'), ('device_

In [ ]:
'customer_locale','buyer_accepts_marketing','contact_email'